<a href="https://colab.research.google.com/github/yamadashamoji/00_tools/blob/main/050_ImageFilterTool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---

## ✅ 使用方法（Colab上）

1. 画像をアップロード  
2. フィルターを選ぶ  
3. （必要に応じて）ぼかし強さをスライダーで調整  
4. 「フィルター適用」ボタンを押す  
5. 比較表示と「加工画像を保存」ボタンが出現  

---

In [ ]:
# 必要なライブラリのインストール（初回のみ）
!pip install -q ipywidgets
!pip install -q opencv-python-headless


In [6]:
# ライブラリの読み込み
import cv2
import numpy as np
from PIL import Image
from io import BytesIO
import matplotlib.pyplot as plt
from IPython.display import display
import ipywidgets as widgets
from google.colab import files
import tempfile

# アップロード用ウィジェット
upload_btn = widgets.FileUpload(accept='image/*', multiple=False)

# フィルター選択ウィジェット
filter_dropdown = widgets.Dropdown(
    options=['グレースケール', 'ぼかし', 'セピア', 'エッジ検出'],
    description='フィルター:'
)

# パラメータ調整スライダー（ぼかし用）
blur_slider = widgets.IntSlider(value=5, min=1, max=25, step=2, description='ぼかし強さ:')

# 適用ボタン
apply_button = widgets.Button(description='フィルター適用')

# グローバル変数
original_img = None
processed_img = None


In [3]:
# フィルター処理関数の定義

def apply_filter(image, filter_name, blur_value=5):
    img = np.array(image.convert('RGB'))
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

    if filter_name == 'グレースケール':
        filtered = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        filtered = cv2.cvtColor(filtered, cv2.COLOR_GRAY2BGR)

    elif filter_name == 'ぼかし':
        k = blur_value if blur_value % 2 == 1 else blur_value + 1  # 奇数に
        filtered = cv2.GaussianBlur(img, (k, k), 0)

    elif filter_name == 'セピア':
        kernel = np.array([[0.272, 0.534, 0.131],
                           [0.349, 0.686, 0.168],
                           [0.393, 0.769, 0.189]])
        filtered = cv2.transform(img, kernel)
        filtered = np.clip(filtered, 0, 255)

    elif filter_name == 'エッジ検出':
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        edges = cv2.Canny(gray, 100, 200)
        filtered = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)

    else:
        filtered = img

    filtered_rgb = cv2.cvtColor(filtered.astype(np.uint8), cv2.COLOR_BGR2RGB)
    return Image.fromarray(filtered_rgb)


In [7]:
# 表示＆保存

def show_images(original, filtered):
    fig, axs = plt.subplots(1, 2, figsize=(10, 5))
    axs[0].imshow(original)
    axs[0].set_title("元画像")
    axs[0].axis('off')

    axs[1].imshow(filtered)
    axs[1].set_title("加工後画像")
    axs[1].axis('off')

    plt.show()

def on_apply_clicked(b):
    global original_img, processed_img

    if upload_btn.value:
        # アップロード画像を取得
        file_info = next(iter(upload_btn.value.values()))
        original_img = Image.open(BytesIO(file_info['content'])).convert('RGB')

        # フィルター適用
        selected_filter = filter_dropdown.value
        blur_strength = blur_slider.value
        processed_img = apply_filter(original_img, selected_filter, blur_strength)

        # 表示
        show_images(original_img, processed_img)

        # 保存用ボタン表示
        display(save_button)

# フィルター適用ボタンにコールバック登録
apply_button.on_click(on_apply_clicked)

# 保存ボタン
save_button = widgets.Button(description="加工画像を保存")

def on_save_clicked(b):
    if processed_img:
        ext = format_dropdown.value.lower()  # 'png' or 'jpeg'
        filename = filename_text.value.strip()

        # ファイル名が空ならデフォルト名
        if not filename:
            filename = "filtered_image"

        # 拡張子をつける
        full_filename = f"{filename}.{ext}"

        # 一時ファイルとして保存 → ダウンロード
        with tempfile.NamedTemporaryFile(delete=False, suffix=f".{ext}") as tmp:
            processed_img.save(tmp.name, format=format_dropdown.value)
            files.download(tmp.name)


save_button.on_click(on_save_clicked)


In [9]:
# 保存ファイル名入力欄
filename_text = widgets.Text(
    value='filtered_image',
    description='ファイル名:'
)

# ファイル形式選択
format_dropdown = widgets.Dropdown(
    options=['PNG', 'JPEG'],
    value='PNG',
    description='形式:'
)


In [ ]:
# ウィジェットを表示して使う

display(widgets.VBox([
    widgets.Label("画像をアップロードしてください："),
    upload_btn,
    filter_dropdown,
    blur_slider,
    apply_button
]))

save_ui = widgets.VBox([
    filename_text,
    format_dropdown,
    save_button
])
display(save_ui)


In [11]:
# 幅・高さ指定用スライダー
width_slider = widgets.IntSlider(value=0, min=0, max=2000, step=10, description='幅(px):')
height_slider = widgets.IntSlider(value=0, min=0, max=2000, step=10, description='高さ(px):')

resize_note = widgets.Label("※ 幅・高さどちらかが0なら元画像サイズを維持")


In [ ]:
def on_save_clicked(b):
    if processed_img:
        ext = format_dropdown.value.lower()  # 'png' or 'jpeg'
        filename = filename_text.value.strip()

        # ファイル名が空ならデフォルト名
        if not filename:
            filename = "filtered_image"

        full_filename = f"{filename}.{ext}"

        # 幅・高さ取得（0なら元画像サイズ）
        width = width_slider.value
        height = height_slider.value
        img_to_save = processed_img

        if width > 0 and height > 0:
            img_to_save = processed_img.resize((width, height))
        elif width > 0:
            aspect = processed_img.height / processed_img.width
            height = int(width * aspect)
            img_to_save = processed_img.resize((width, height))
        elif height > 0:
            aspect = processed_img.width / processed_img.height
            width = int(height * aspect)
            img_to_save = processed_img.resize((width, height))
        # どちらも0 → リサイズなし

        # 一時ファイル保存 → ダウンロード
        with tempfile.NamedTemporaryFile(delete=False, suffix=f".{ext}") as tmp:
            img_to_save.save(tmp.name, format=format_dropdown.value)
            files.download(tmp.name)


In [13]:
save_ui = widgets.VBox([
    filename_text,
    format_dropdown,
    width_slider,
    height_slider,
    resize_note,
    save_button
])

In [ ]:
display(save_ui)

### ✨ 最終的な操作イメージ

1. フィルターを適用  
2. ファイル名・保存形式を選択  
3. 幅・高さを指定（片方だけでもOK、どちらも0で元サイズ）  
4. 「加工画像を保存」をクリック  
5. 指定のサイズ＆形式で保存される！

まずは「画像にフィルターをかけるツール」の機能と設計を明確にし、その後に**移植も意識した仕様書**を作成します。

---

## 🔧 ツールの基本仕様

### 📌 ツール名（仮）  
**ImageFilterTool**

### 📍 開発環境（初期）  
- 言語: Python 3.x  
- 実行環境: Google Colab  
- ライブラリ: OpenCV, Pillow, NumPy, IPython Widgets（UI用）

---

## 🎯 機能一覧（初期）

| 機能 | 内容 |
|------|------|
| 画像アップロード | ユーザーがローカル画像をColabにアップロード |
| フィルター適用 | 複数のエフェクト（例：グレースケール、ぼかし、セピア、エッジ検出など）を選択して画像に適用 |
| 比較表示 | 元画像と加工画像を並べて表示 |
| 画像保存 | 加工後の画像をダウンロード可能に |
| UI操作 | ドロップダウンやスライダーによるフィルター選択やパラメータ調整 |

---

## 💻 デスクトップ版への移植方針

| 項目 | 内容 |
|------|------|
| 実行環境 | Python + PyQt5 または Tkinter、もしくは Electron + Pythonバックエンド |
| オフライン対応 | ライブラリはpipでインストール可能な範囲に制限（例：OpenCV, Pillow） |
| UI設計 | Google Colabでの操作性をベースに、デスクトップでも類似UIを実現 |

---

## 📄 仕様書（ドラフト）

---

### 1. 概要  
本ツールは、ユーザーがアップロードした画像に対し、選択したフィルターを適用し、加工済み画像の保存ができるツールである。最初はGoogle Colab上で動作し、最終的にはデスクトップアプリとして移植する。

---

### 2. 機能詳細

#### 2.1 画像アップロード  
- 入力形式: JPG, PNG, BMP  
- UI: `file upload` ウィジェットを使用（Colab）  
- 保存先: 一時的にColab上に保存  

#### 2.2 フィルター適用  
- 利用可能なフィルター（予定）:
  - グレースケール
  - ぼかし（ガウシアン、平均）
  - セピア調
  - エッジ検出（Canny）
  - 反転（ネガ）
  - 輪郭強調（Laplacian）

- 選択方式: ドロップダウン形式  
- パラメータ調整（ぼかしの強度など）: スライダーで調整可能

#### 2.3 比較表示  
- 元画像と加工後画像を横並びで表示（`matplotlib`または`cv2.hconcat`利用）

#### 2.4 画像保存  
- フォーマット: PNGで保存（Colab内に保存 or ローカルにダウンロードリンク）  

#### 2.5 UI構成（Colab）  
- ファイルアップローダー  
- フィルター選択ドロップダウン  
- パラメータ調整用スライダー  
- 「適用」ボタン  
- 「保存」ボタン  

---

### 3. 将来的な移植仕様（デスクトップ）

| 要素 | 設計意図 |
|------|----------|
| UI | PyQt5 で再現性の高いGUIに移植（QComboBox, QSlider, QPushButtonなど） |
| 処理ロジック | Colab版のフィルター処理関数をそのまま流用 |
| ファイル処理 | `QFileDialog` による画像の読み書き |
| 表示 | `QLabel` + `QPixmap`を使って画像を表示 |

---

### 4. 使用ライブラリ（初期）

- `opencv-python`
- `Pillow`
- `numpy`
- `matplotlib`
- `IPython.display`
- `ipywidgets`

---